# Scenario 2: A cross-functional team with one data scientist working on an ML model

MLflow setup:

- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem
- The experiments can be explored locally by accessing the local tracking server.

In [1]:
import mlflow

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/kylepaul/notebooks/mlops-zoom-camp-2022/session_2/mlruns/2', creation_time=1685474424412, experiment_id='2', last_update_time=1685474424412, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/home/kylepaul/notebooks/mlops-zoom-camp-2022/session_2/mlruns/1', creation_time=1685346804195, experiment_id='1', last_update_time=1685346804195, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1685346718761, experiment_id='0', last_update_time=1685346718761, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():
    X, y = load_iris(return_X_y=True)
    
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)
    
    model_lr = LogisticRegression(**params).fit(X,y)
    y_pred = model_lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    
    mlflow.sklearn.log_model(model_lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 'mlflow-artifacts:/3/132f26887702465291c8d6b7afc5901d/artifacts'


/home/kylepaul/anaconda3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1685482274169, experiment_id='3', last_update_time=1685482274169, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/home/kylepaul/notebooks/mlops-zoom-camp-2022/session_2/mlruns/2', creation_time=1685474424412, experiment_id='2', last_update_time=1685474424412, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/home/kylepaul/notebooks/mlops-zoom-camp-2022/session_2/mlruns/1', creation_time=1685346804195, experiment_id='1', last_update_time=1685346804195, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1685346718761, experiment_id='0', last_update_time=1685346718761, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [10]:
from mlflow import MlflowClient
client = MlflowClient("http://127.0.0.1:5000")

In [11]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1685475228645, description='', last_updated_timestamp=1685476846414, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685475228716, current_stage='Production', description='The model version 2 was transitioned to Staging on 2023-05-31', last_updated_timestamp=1685476846414, name='nyc-taxi-regressor', run_id='8264fb3eb8fb47f69ce47e08af58922b', run_link='', source='/home/kylepaul/notebooks/mlops-zoom-camp-2022/session_2/mlruns/1/8264fb3eb8fb47f69ce47e08af58922b/artifacts/models_mlflow', status='READY', status_message='', tags={}, user_id='', version='1'>,
  <ModelVersion: aliases=[], creation_timestamp=1685475316872, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2023-05-31', last_updated_timestamp=1685475758439, name='nyc-taxi-regressor', run_id='cf1747fba5e24f5d95d856cdc9c569a3', run_link='', source='/home/kylepaul/notebooks/mlops-zoom-camp-2022/session_2/mlruns/1/cf1747fba5e24f5d

In [27]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids = '1',
    filter_string ="",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
)

In [28]:
for run in runs:
    print(f"run id: {run.info.run_id}")

run id: 9b694d07a0314eaf96571255af55dba6
run id: 8264fb3eb8fb47f69ce47e08af58922b
run id: 00fc951f5f76459a81d53127a1ee73f5
run id: c2463632b62347f8bba2729c5d9cf124
run id: 07f74ce7cc684ca2983a32fea90f54ee


In [29]:
run_id = "132f26887702465291c8d6b7afc5901d"
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2023/05/31 04:46:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1685483187677, current_stage='None', description='', last_updated_timestamp=1685483187677, name='iris-classifier', run_id='132f26887702465291c8d6b7afc5901d', run_link='', source='mlflow-artifacts:/3/132f26887702465291c8d6b7afc5901d/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>